In [ ]:
import pandas as pd
import requests
import plotly.express as px
from IPython.display import display
import ipywidgets as widgets

# 1. Fetch Global COVID-19 Summary Data
def get_global_summary():
    url = "https://disease.sh/v3/covid-19/all"
    response = requests.get(url)
    data = response.json()
    return data

# 2. Fetch Country-wise COVID-19 Data
def get_country_data():
    url = "https://disease.sh/v3/covid-19/countries"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)

# 3. Fetch Time-series Data for a Country
def get_country_timeseries(country):
    url = f"https://disease.sh/v3/covid-19/historical/{country}?lastdays=all"
    response = requests.get(url)
    data = response.json()
    if 'timeline' in data:
        df = pd.DataFrame(data['timeline'])
        df = df.T  # Transpose the data so dates become rows
        df.index = pd.to_datetime(df.index)
        df.columns = ['Cases', 'Deaths', 'Recovered']
        df.sort_index(inplace=True)
        return df
    else:
        return pd.DataFrame()

# 4. Display Global Summary
def display_global_summary():
    summary = get_global_summary()
    print(" Global COVID-19 Summary:")
    print(f"Total Cases     : {summary['cases']:,}")
    print(f"Total Deaths    : {summary['deaths']:,}")
    print(f"Total Recovered : {summary['recovered']:,}")
    print(f"Active Cases    : {summary['active']:,}")
    print(f"Today Cases     : {summary['todayCases']:,}")
    print(f"Today Deaths    : {summary['todayDeaths']:,}")
    print(f"Today Recovered : {summary['todayRecovered']:,}")

# 5. Display Country Table
def display_country_table(df):
    display(df[['country', 'cases', 'deaths', 'recovered', 'active']].sort_values(by='cases', ascending=False).reset_index(drop=True))

# 6. Create Interactive Country Explorer
def country_explorer(df):
    countries = df['country'].sort_values().tolist()
    
    dropdown = widgets.Dropdown(
        options=countries,
        description='Country:',
        layout={'width': '300px'}
    )
    
    output = widgets.Output()
    
    def on_change(change):
        output.clear_output()
        if change['type'] == 'change' and change['name'] == 'value':
            with output:
                country = change['new']
                print(f" {country} COVID-19 Time Series")
                ts = get_country_timeseries(country)
                if not ts.empty:
                    fig = px.line(ts, y=['Cases', 'Deaths', 'Recovered'], title=f"{country} COVID-19 Time Series")
                    fig.update_layout(autosize=False, width=800, height=400)
                    fig.show()
                else:
                    print("No data available for this country.")
    
    dropdown.observe(on_change)
    
    display(dropdown, output)

# Main Execution Flow
# Step-by-step execution with comments for each block
print(" Fetching Data...")
df_countries = get_country_data()         # Load country-level data
display_global_summary()                  # Print global summary
print("\n Top Countries Overview:")
display_country_table(df_countries)       # Show top countries in table
print("\n Explore Specific Country:")
country_explorer(df_countries)            # Launch interactive explorer
